Import required library

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))          # Normalize to range [-1, 1]
])

In [20]:
train_data=datasets.MNIST('data1',train=True,download=True,transform=transform)
test_data=datasets.MNIST('data1',train=False,download=True,transform=transform)


Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.3MB/s]


Extracting data1/MNIST/raw/train-images-idx3-ubyte.gz to data1/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 28.9k/28.9k [00:00<00:00, 488kB/s]


Extracting data1/MNIST/raw/train-labels-idx1-ubyte.gz to data1/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.42MB/s]


Extracting data1/MNIST/raw/t10k-images-idx3-ubyte.gz to data1/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.89MB/s]

Extracting data1/MNIST/raw/t10k-labels-idx1-ubyte.gz to data1/MNIST/raw



In [24]:
train_data = datasets.MNIST(
    root = 'data1',
    train = True,
    transform = transform,

)

test_data = datasets.MNIST(
    root = 'data1',
    train = False,
    transform = transform,

)

In [26]:
train_data , test_data

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data1
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5,), std=(0.5,))
            ),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: data1
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5,), std=(0.5,))
            ))

In [31]:
from torch.utils.data import DataLoader

loaders = {
    'train' : DataLoader(train_data,
                         batch_size=100,
                         shuffle=True,
                         num_workers=1),
    'test' : DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)
}

In [36]:
loaders['test']

In [33]:
class BestDigitClassifier(nn.Module):
    def __init__(self):
        super(BestDigitClassifier, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),                    # Add BatchNorm
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),                    # Add BatchNorm
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.dropout = nn.Dropout(0.5)            # Add Dropout
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x



In [35]:
model = BestDigitClassifier()
criterion = nn.CrossEntropyLoss()          # Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer



In [37]:
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in loaders['test']:
        optimizer.zero_grad()          # Gradients clear karna
        outputs = model(images)        # Forward pass
        loss = criterion(outputs, labels)  # Loss calculate karna
        loss.backward()                # Backward pass
        optimizer.step()               # Weights update karna

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(loaders['test']):.4f}")


Epoch [1/5], Loss: 0.5640
Epoch [2/5], Loss: 0.1617
Epoch [3/5], Loss: 0.1264
Epoch [4/5], Loss: 0.1043
Epoch [5/5], Loss: 0.0786


In [38]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in loaders['test']:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Highest score wala class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 99.40%
